In [1]:
import numpy as np
import os
import pandas as pd
from tqdm import tqdm 
import cv2
from keras.models import load_model
import warnings
warnings.simplefilter('ignore')
image_size = 480

In [2]:
enc_to_names = ['bacterial_leaf_blight', 'bacterial_leaf_streak', 'bacterial_panicle_blight', 'blast', 'brown_spot', 'dead_heart', 'downy_mildew', 'hispa', 'normal', 'tungro']
test_dir = '../input/paddy-disease-classification/test_images/'
model = load_model('../input/resnet-incepv3-version12-training/best_model (1).h5')

2022-08-14 04:01:49.206897: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 04:01:49.358537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 04:01:49.359338: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 04:01:49.361305: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [3]:
img = cv2.imread('../input/paddy-disease-classification/test_images/200001.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (image_size, image_size), interpolation = cv2.INTER_AREA)
img = img.reshape(1, image_size, image_size, 3)
pred = model.predict(img)
print(pred[0])
y = np.argmax(pred[0])
print(y)
print(enc_to_names[y])

2022-08-14 04:02:06.622179: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-08-14 04:02:10.632698: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


[5.6924088e-12 1.4582366e-15 4.1387122e-13 1.4420770e-09 2.4173874e-10
 6.5799821e-10 4.5041730e-09 9.9999630e-01 3.6851163e-06 4.1816399e-09]
7
hispa


In [4]:
sample_submission = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')
sample_submission.head()

,image_id,label
0,200001.jpg,NaN
1,200002.jpg,NaN
2,200003.jpg,NaN
3,200004.jpg,NaN
4,200005.jpg,NaN


In [5]:
submission_data = []
temp = None
for img_id in tqdm(os.listdir(test_dir)):
    path = os.path.join(test_dir, img_id)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (image_size, image_size), interpolation = cv2.INTER_AREA)
    img = img.reshape(1, image_size, image_size, 3)
    pred = model.predict(img)
    pred = np.argmax(pred[0])
    pred = enc_to_names[pred]
    temp = {'image_id': img_id, 'label': pred}
    submission_data.append(temp)

100%|██████████| 3469/3469 [05:25<00:00, 10.66it/s]


In [6]:
submission = pd.DataFrame(submission_data)
submission.to_csv('submission.csv', index=False)